# Mount drive and append path to PYTONPATH


In [ ]:
import os
import sys

from google.colab import drive

drive.mount("/content/drive")
sys.path.append("/content/drive/MyDrive/DeepLCMS/train_google_colab")

# Import and install libraries

In [ ]:
%%capture
!pip install lightning
!pip install timm
!pip install torchinfo

In [ ]:
import gc
from typing import Optional, Tuple

import colab_functions
import colab_utils
import pandas as pd
import prepare_data
import pytorch_lightning as pl
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchinfo
import train_NN
from google.colab import drive
from lightning.pytorch.loggers import CSVLogger
from pytorch_lightning import LightningModule
from pytorch_lightning.callbacks import Callback, EarlyStopping
from pytorch_lightning.trainer.trainer import Trainer
from timm import create_model
from torchmetrics.classification import (
    BinaryAUROC,
    BinaryF1Score,
    BinaryPrecision,
    BinaryRecall,
)

In [ ]:
# Set the CUDA_VISIBLE_DEVICES environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Unzip data

In [ ]:
!unzip -q experiment.zip

# Check if GPU is used

In [ ]:
device = colab_functions.get_device()

# Findings the best version of tf_efficientnetv2


In [ ]:
tf_efficientnetv2_models = timm.list_models("resnet*", pretrained=True)[:2]


for pretrained_model in tf_efficientnetv2_models:
    try:
        temp_model = train_NN.PretrainedModelEvaluator(pretrained_model)

        (
            preprocess_train,
            preprocess_val,
            preprocess_test,
        ) = prepare_data.get_timm_transforms(temp_model)

        (
            train_dataloader,
            val_dataloader,
            test_dataloader,
        ) = prepare_data.get_dataloaders(
            preprocess_train=preprocess_train,
            preprocess_val=preprocess_val,
            preprocess_test=preprocess_test,
        )

        # metrics_callback = train_NN.MetricsCallback()
        logger = CSVLogger("logs", name=str(pretrained_model))

        # trainer = pl.Trainer(max_epochs=1, callbacks=[metrics_callback], log_every_n_steps=1)
        trainer = Trainer(
            max_epochs=50,
            log_every_n_steps=1,
            logger=logger,
            callbacks=[EarlyStopping(monitor="val_loss", mode="min")],
        )

        trainer.fit(
            model=temp_model,
            train_dataloaders=train_dataloader,
            val_dataloaders=val_dataloader,
        )

        # Clean up resources
        resources_to_delete = [
            temp_model,
            preprocess_train,
            preprocess_val,
            preprocess_test,
            train_dataloader,
            val_dataloader,
            test_dataloader,
            trainer,
        ]

        gc.collect()
    except RuntimeError as e:
        pass

results_df = colab_functions.get_experiment_results()
results_df.to_csv("resnet_models_results.csv", index=False)
colab_functions.plot_experiment_results(results_df)

# Finding the best learining rate

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir='/content/lightning_logs'